In [53]:
import osmnx as ox
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import requests 
import csv

In [42]:
def count_objects_in_polygon(polygon):
    if not isinstance(polygon, (Polygon, MultiPolygon)):
        raise TypeError("Not polygon obj")

    tags = {
        'aeroway': 'aerodrome',
        'seamark:type': 'harbour',
        'highway': 'motorway' #if we set this to True, we will get all roads, this is just highways
    }

    try:

        objects = ox.features_from_polygon(polygon, tags)
    except ox._errors.InsufficientResponseError: #specific error is InsufficientResponseError, but osmnx doesn't allow you to import it for some reason
        return {
        'airports': 0,
        'harbors': 0,
        'highways': 0
    }

    num_airports = len(objects[objects['aeroway'] == 'aerodrome']) if 'aeroway' in objects.columns else 0
    num_harbors = len(objects[objects['seamark:type'] == 'harbour']) if 'seamark:type' in objects.columns else 0
    num_highways = len(objects[objects['highway'].notna()]) if 'highway' in objects.columns else 0


    return {
        'airports': num_airports,
        'harbors': num_harbors,
        'highways': num_highways
    }



In [43]:

def bounding_coordinates(place, country): # this is from the vegetation notebook, the current one in the repo is corupt. once its not corupt we can import this func
    url = f"https://nominatim.openstreetmap.org/search.php?q={place}+{country}&polygon_geojson=1&format=json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        polygon_coordinates = 0
        bounding_coordinates = 0
        if(data[0]["class"] != "boundary"):
          for i in range(len(data)):
            if(data[i]["class"] == "boundary"):
              polygon_coordinates = data[i]['geojson']['coordinates'][0]
        else:
          polygon_coordinates = data[0]['geojson']['coordinates'][0]
        bounding_coordinates = [float(i) for i in data[0]['boundingbox']]
        return [polygon_coordinates, bounding_coordinates]

In [44]:
def flatten_coords(coord_list): #ChatGPT generated FYI, sorry ): , had to run to class
    """ Flatten the list of coordinates to a list of tuples. """
    # This function should handle nested structures appropriately
    if isinstance(coord_list[0], list):
        # If the first item is a list, we assume it's a list of lists
        if isinstance(coord_list[0][0], list):
            # If the nested list also contains a list, recurse deeper
            return flatten_coords([item for sublist in coord_list for item in sublist])
        else:
            # Otherwise, it's just one level deep of coordinate pairs
            return [tuple(item) for item in coord_list]
    else:
        # Base case: if we're at the coordinate pair level, stop flattening
        return [tuple(coord_list)]


In [45]:

def return_weight(city, country):
    poly_coords = bounding_coordinates(city, country)[0]
    poly_coords = flatten_coords(poly_coords)
    poly = Polygon(poly_coords)


    object_counts = count_objects_in_polygon(poly)
   

    square_surface_area = poly.area


    totalSum = 5*(object_counts['airports']) + 3*(object_counts['harbors']) + (object_counts['highways'])

    return totalSum/square_surface_area

In [49]:
def test():
    print(return_weight("D", "Mozambique"))


In [60]:

def update_transport_network_density(csv_filepath):
    # Read the CSV data
    with open(csv_filepath, newline='') as file:
        reader = csv.DictReader(file)
        rows = list(reader)  # Convert iterator to list to reuse it

    for row in rows:
        print("Checking:", row['Region'].replace("_", " "), row['Country'].replace("_", " "))
        result = return_weight(row['Region'].replace("_", " "), row['Country'].replace("_", " "))
        print(row['Region'], row['Country'], result)
        row['Transport_Network_Density'] = float(row['Transport_Network_Density']) + result

    with open(csv_filepath, 'w', newline='') as file:
        fieldnames = rows[0].keys()  # Grab the field names from the first row
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

In [62]:
update_transport_network_density("/Users/alexforman/Downloads/foodwatchInput.csv")

Checking: Bamingui Central African Republic


/opt/homebrew/lib/python3.12/site-packages/osmnx/_overpass.py:245: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Bamingui Central_African_Republic 4.221284958208782
Checking: Bangui Central African Republic
Bangui Central_African_Republic 887.2595543923152
Checking: Badakhshan Afghanistan


/opt/homebrew/lib/python3.12/site-packages/osmnx/_overpass.py:245: UserWarning: This area is 34 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Badakhshan Afghanistan 10.228901654130443
Checking: Badghis Afghanistan
Badghis Afghanistan 2.4421677548779237
